In [ ]:
# Install dependencies if not already installed:
# pip install openai-whisper sounddevice scipy deep-translator gTTS simpleaudio numpy

import whisper
import sounddevice as sd
from scipy.io.wavfile import write
from deep_translator import GoogleTranslator
from gtts import gTTS
import simpleaudio as sa
import tempfile
import numpy as np
import queue
import threading
import os

# -------------------------------
# 1. Load Whisper Model
# -------------------------------
print("Loading Whisper model (small for faster real-time)...")
model = whisper.load_model("small")

# -------------------------------
# 2. Setup real-time audio
# -------------------------------
fs = 16000  # Sample rate
chunk_duration = 3  # seconds per audio chunk
blocksize = int(fs * chunk_duration)

audio_queue = queue.Queue()

def audio_callback(indata, frames, time, status):
    if status:
        print("Audio status:", status)
    # Copy and queue the audio chunk
    audio_queue.put(indata.copy())

# -------------------------------
# 3. Processing thread
# -------------------------------
def process_audio():
    while True:
        audio_chunk = audio_queue.get()
        if audio_chunk is None:  # Exit signal
            break
        # Save chunk to temporary WAV
        temp_wav = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)
        write(temp_wav.name, fs, audio_chunk)
        
        # 3a. Transcribe
        result = model.transcribe(temp_wav.name, language='en')
        english_text = result["text"].strip()
        if english_text:
            print("English:", english_text)
            
            # 3b. Translate
            telugu_text = GoogleTranslator(source='en', target='te').translate(english_text)
            print("Telugu:", telugu_text)
            
            # 3c. Text-to-Speech
            tts = gTTS(telugu_text, lang='te')
            temp_mp3 = tempfile.NamedTemporaryFile(suffix=".mp3", delete=False)
            tts.save(temp_mp3.name)
            
            # Play audio
            try:
                wave_obj = sa.WaveObject.from_wave_file(temp_mp3.name)
                play_obj = wave_obj.play()
                play_obj.wait_done()
            except Exception as e:
                print("Error playing audio:", e)
            
            # Clean up temp files
            os.remove(temp_wav.name)
            os.remove(temp_mp3.name)

# Start processing thread
thread = threading.Thread(target=process_audio, daemon=True)
thread.start()

# -------------------------------
# 4. Start real-time microphone stream
# -------------------------------
try:
    print("Listening in real-time... Speak something!")
    with sd.InputStream(samplerate=fs, channels=1, dtype='int16',
                        callback=audio_callback, blocksize=blocksize):
        import time
        while True:
            time.sleep(0.1)
except KeyboardInterrupt:
    print("\nStopping...")
finally:
    # Stop the processing thread
    audio_queue.put(None)
    thread.join()
